### Функция ошибки (с регуляризацией)

Основная функция ошибки для одного элемента матрицы взаимодействий $ A_{ij} $:
$$
E_{ij} = (A_{ij} - U_i^T V_j)^2 + \lambda (||U_i||^2 + ||V_j||^2)
$$
Здесь:
- $ A_{ij} $ — фактическое взаимодействие (например, рейтинг).
- $ U_i $ и $ V_j $ — вектора скрытых факторов для пользователя $ i $ и товара $ j $, соответственно.
- $ U_i^T V_j $ — предсказанное взаимодействие.
- $ \lambda $ — коэффициент регуляризации для предотвращения переобучения.

### Вычисление градиента для каждого параметра

1. **Градиент по $ U_i $**

   Нам нужно минимизировать функцию ошибки $ E_{ij} $, изменяя $ U_i $. Для этого вычисляется частная производная ошибки по каждому элементу вектора $ U_i $:

   $$
   \frac{\partial E_{ij}}{\partial U_i} = -2 (A_{ij} - U_i^T V_j) V_j + 2 \lambda U_i
   $$
   - $ -2 (A_{ij} - U_i^T V_j) V_j $ — это градиент без регуляризации, который корректирует вектор $ U_i $ на основе ошибки предсказания.
   - $ 2 \lambda U_i $ — регуляризация, которая уменьшает значения элементов $ U_i $, чтобы избежать переобучения.

   Итоговое обновление $ U_i $ с использованием градиентного спуска:
   $$
   U_i \gets U_i + \alpha \left( (A_{ij} - U_i^T V_j) V_j - \lambda U_i \right)
   $$
   Здесь $ \alpha $ — скорость обучения (learning rate).

   **Аналогично, градиент по $ V_j $:**
   $$
   V_j \gets V_j + \alpha \left( (A_{ij} - U_i^T V_j) U_i - \lambda V_j \right)
   $$

2. **Градиенты по $b_u$, $b_i$**:
   
     $$
     b_u[i] \leftarrow b_u[i] + \alpha \left( e_{ij} - \lambda b_u[i] \right)
     $$

     $$
     b_i[j] \leftarrow b_i[j] + \alpha \left( e_{ij} - \lambda b_i[j] \right)
     $$

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Загрузка данных
df = pd.read_csv('data/ml-latest-small/ratings.csv')
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()

# Создание индексов для пользователей и фильмов
user_ids = df['userId'].astype('category').cat.codes.values
item_ids = df['movieId'].astype('category').cat.codes.values

df['user_idx'] = user_ids
df['item_idx'] = item_ids

# Разделение данных на тренировочные и тестовые наборы
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Матрица рейтингов
def create_matrix(df, n_users, n_items):
    matrix = np.zeros((n_users, n_items))
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating
    return matrix

R_train = create_matrix(train_df, n_users, n_items)
R_test = create_matrix(test_df, n_users, n_items)

In [10]:
# Функция для расчета RMSE
def calculate_rmse(R_test, model):
    xs, ys = R_test.nonzero()
    predicted = []
    actual = []
    progress_bar = tqdm(total=len(xs), desc="Calculating RMSE")
    for x, y in zip(xs, ys):
        predicted.append(model.predict(x, y))
        actual.append(R_test[x, y])
        progress_bar.update(1)
    progress_bar.close()
    return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

In [11]:
class SVD:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R > 0])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        progress_bar = tqdm(total=self.iterations, desc="Training Progress")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}"})
            progress_bar.update(1)
        progress_bar.close()

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.predict(i, j)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление скрытых факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * self.P[i, :] - self.beta * self.Q[j, :])

    def predict(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

In [12]:
class SVDPlusPlus:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K  # Количество скрытых факторов
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations

        # Инициализация параметров
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))  # Пользовательские факторы
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))  # Товарные факторы
        self.y = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))  # Латентные векторы для товаров

        self.b_u = np.zeros(self.num_users)  # Биасы пользователей
        self.b_i = np.zeros(self.num_items)  # Биасы товаров
        self.b = np.mean(self.R[self.R > 0])  # Глобальный биас

        # Предварительное сохранение индексов непустых оценок
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Предварительное вычисление множества N_u для каждого пользователя
        self.N_u = {i: [j for j in range(self.num_items) if self.R[i, j] > 0] for i in range(self.num_users)}

    def train(self):
        progress_bar = tqdm(total=self.iterations, desc="Training SVD++")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}"})
            progress_bar.update(1)
        progress_bar.close()

    def sgd(self):
        y_update = np.zeros((self.num_items, self.K))  # Для обновления латентных векторов y_j
        
        for i, j, r in self.samples:
            # Имплицитный фидбек для пользователя i
            N_u = self.N_u[i]
            if len(N_u) > 0:
                implicit_feedback = np.sum(self.y[N_u], axis=0) / np.sqrt(len(N_u))
            else:
                implicit_feedback = np.zeros(self.K)

            # Предсказание
            prediction = self.b + self.b_u[i] + self.b_i[j] + np.dot(self.P[i, :] + implicit_feedback, self.Q[j, :].T)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * (self.P[i, :] + implicit_feedback) - self.beta * self.Q[j, :])

            # Обновление латентных факторов y для каждого товара в N_u
            if len(N_u) > 0:
                y_update[N_u] += self.alpha * (error / np.sqrt(len(N_u)) * self.Q[j, :] - self.beta * self.y[N_u])

        # Обновление латентных векторов y для всех товаров после прохода по данным
        self.y += y_update

    def predict(self, i, j):
        N_u = self.N_u[i]
        if len(N_u) > 0:
            implicit_feedback = np.sum(self.y[N_u], axis=0) / np.sqrt(len(N_u))
        else:
            implicit_feedback = np.zeros(self.K)
        return self.b + self.b_u[i] + self.b_i[j] + np.dot(self.P[i, :] + implicit_feedback, self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))


In [16]:
K=5
iterations=25
alpha=0.005
beta=0.01

# Запуск моделей
svd = SVD(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
svd.train()
rmse_test = calculate_rmse(R_test, svd)
print(f"Test RMSE: {rmse_test:.4f}")

svdpp = SVDPlusPlus(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
svdpp.train()
rmse_test = calculate_rmse(R_test, svdpp)
print(f"Test RMSE: {rmse_test:.4f}")

Calculating RMSE: 100%|██████████| 30251/30251 [00:00<00:00, 205788.34it/s]


Test RMSE: 0.8828


Calculating RMSE: 100%|██████████| 30251/30251 [00:02<00:00, 14202.34it/s]

Test RMSE: 0.9165
